In [ ]:
5#线性

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import random
import types

In [7]:
def loaddataset(filename):
    datamat=[]
    ladelmat=[]
    fr=open(filename)
    for line in fr.readlines():
        linearr=line.strip().split('\t')
        datamat.append([float(linearr[0]),float(linearr[1])])
        ladelmat.append([float(linearr[2])])
    return datamat,ladelmat

In [8]:
def selectj(i,m):
    j=i
    while j==i:
        j=int(random.uniform(0,m))
    return j

In [24]:
def clipalpha(aj,H,L):
    if aj>H:
        return H
    if aj<L:
        return L
    else:
        return aj

In [26]:
def smo(datamat,labelmat,C,toler,maxIter):
    datamat=np.mat(datamat)
    labelmat=np.mat(labelmat)
    m,n=np.shape(datamat)
    b=0
    alphas=np.zeros(m,1)
    iter_num=0
    while iter_num<maxIter:
        alphas_pairs_change=0
        for i in range(m):
            W=np.dot(np.multiply(labelmat,alphas).T,datamat)
            fw_i=np.dot(datamat[i],W)
            E_i=fw_i-labelmat[i]
            if (labelmat[i]*E_i>toler and alphas[i]>0) or (labelmat[i]*E_i<-toler and alphas[i]<C):
                j=selectj(i,m)
                fw_j=np.dot(datamat[j],W)
                E_j=fw_j=labelmat[j]
                alpha_i_old=alphas[i]
                alpha_j_old=alphas[j]
            if labelmat[i]!=labelmat[j]:
                L=max(0,alphas[j]-alphas[i])
                H=min(C,C+alphas[j]-alphas[i])
            else:
                L=max(0,alphas[i]+alphas[j]-C)
                H=min(C,alphas[i]+alphas[j])
            if L==H:
                continue
            eta=2.0*datamat[i,:]*datamat[j,:]-datamat[i,:]*datamat[i,:].T-datamat[j,:]*datamat[j,:].T
            if eta>=0:
                continue
            alphas[j]-=labelmat[j]*(E_j-E_i)/eta
            alphas[j]=clipalpha(alphas[j],H,L)
            if ads(alphas[j]-alpha_j_old)<0.0001:
                continue
            else:
                alphas[i]=alphas[i]+labelmat[j]*labelmat[i]*(alpha_j_old-alphas[j])
            b1=b-E_i-labelmat[i]*(alpha_i_old-alphas[i])*np.dot(datamat[i,:],datamat[i,:].T)-labelmat[j]*(alpha_j_old-alphas[j])*np.dot(datamat[i,:],datamat[j,:].T)
            b2=b-E_j-labelmat[j]*(alpha_j_old-alphas[j])*np.dot(datamat[j,:],datamat[j,:].T)-labelmat[i]*(alpha_i_old-alphas[i])*np.dot(datamat[i,:],datamat[j,:].T)
            if 0<alphas[i] and alphas[i]<C:
                b=b1
            elif 0<alphas[j] and alphas[j]<C:
                b=b2
            else:
                b=(b1+b2)/2
            alphas_pairs_change+=1
        if alphas_pairs_change==0:
            iter_num+=1
        else:
            iter_num=0
    return b,alphas
        